* Basic text processing and pattern matching
* Building machine learning models with text
* Representing text with word embeddings that numerically capture the meaning of words and documents

Use spacy

In [4]:
import spacy
nlp = spacy.load('en_core_web_sm')

OSError: [E050] Can't find model 'en_core_web_sm'. It doesn't seem to be a shortcut link, a Python package or a valid path to a data directory.

In [3]:
doc = nlp("Tea is healthy and calming, don't you think?")

NameError: name 'nlp' is not defined